# Shares of the 1903 Prize in Physics
You want to examine the laureates of the 1903 prize in physics and how they split the prize. Here is a query without projection:

db.laureates.find_one({"prizes": {"$elemMatch": {"category": "physics", "year": "1903"}}})
Which projection(s) will fetch ONLY the laureates' full names and prize share info? I encourage you to experiment with the console and re-familiarize yourself with the structure of laureate collection documents.

In [1]:
from pymongo import MongoClient

client = MongoClient('mongodb://localhost:27017')

# Connect to the "nobel" database
db = client.nobel

In [2]:
print(db.laureates.find_one({"prizes": {"$elemMatch": {"category": "physics", "year": "1903"}}}))

{'_id': ObjectId('6018024085c61d018cc9850d'), 'id': '4', 'firstname': 'Antoine Henri', 'surname': 'Becquerel', 'born': '1852-12-15', 'died': '1908-08-25', 'bornCountry': 'France', 'bornCountryCode': 'FR', 'bornCity': 'Paris', 'diedCountry': 'France', 'diedCountryCode': 'FR', 'gender': 'male', 'prizes': [{'year': '1903', 'category': 'physics', 'share': '2', 'motivation': '"in recognition of the extraordinary services he has rendered by his discovery of spontaneous radioactivity"', 'affiliations': [{'name': 'École Polytechnique', 'city': 'Paris', 'country': 'France'}]}]}


In [8]:
print(db.laureates.find_one({"prizes":["firstname", "surname", "prizes"]}))

None


In [4]:
print(db.laureates.find_one(["firstname", "surname", "prizes.share"]))

None


In [9]:
print(db.laureates.find_one({"prizes": {"firstname": 1, "surname": 1, "prizes.share": 1, "_id": 0}}))

None


# Rounding up the G.S. crew
In chapter 2, you used a regular expression object Regex to find values that follow a pattern. We can also use the regular expression operator $regex for the same purpose. For example, the following query:

{ "name": {$regex: "^Py"}    }
will fetch documents where the field 'name' starts with "Py". Here the caret symbol ^ means "starts with".

In this exercise, you will use regular expressions, projection, and list comprehension to collect the full names of laureates whose initials are "G.S.".

In [10]:
# Find laureates whose first name starts with "G" and last name starts with "S"
docs = db.laureates.find(
    filter={
        "firstname": {"$regex": "^G"},
        "surname": {"$regex": "^S"}})
# Print the first document 
print(docs[0])

{'_id': ObjectId('6018024085c61d018cc985e8'), 'id': '421', 'firstname': 'George D.', 'surname': 'Snell', 'born': '1903-12-19', 'died': '1996-06-06', 'bornCountry': 'USA', 'bornCountryCode': 'US', 'bornCity': 'Bradford, MA', 'diedCountry': 'USA', 'diedCountryCode': 'US', 'diedCity': 'Bar Harbor, ME', 'gender': 'male', 'prizes': [{'year': '1980', 'category': 'medicine', 'share': '3', 'motivation': '"for their discoveries concerning genetically determined structures on the cell surface that regulate immunological reactions"', 'affiliations': [{'name': 'Jackson Laboratory', 'city': 'Bar Harbor, ME', 'country': 'USA'}]}]}


In [11]:
# Use projection to select only firstname and surname
docs = db.laureates.find(
        filter= {"firstname" : {"$regex" : "^G"},
                 "surname" : {"$regex" : "^S"}  },
	projection=["firstname", "surname"]) 

# Print the first document 
print(docs[0])

{'_id': ObjectId('6018024085c61d018cc985e8'), 'firstname': 'George D.', 'surname': 'Snell'}


In [12]:
# Use projection to select only firstname and surname
docs = db.laureates.find(
       filter= {"firstname" : {"$regex" : "^G"},
                "surname" : {"$regex" : "^S"}  },
   projection= ["firstname", "surname"]  )

# Iterate over docs and concatenate first name and surname
full_names = [doc["firstname"] + " " + doc["surname"] for doc in docs]

# Print the full names
print(full_names)

['George D. Snell', 'Gustav Stresemann', 'Glenn Theodore Seaborg', 'George J. Stigler', 'George F. Smoot', 'George E. Smith', 'George P. Smith', 'George Bernard Shaw', 'Giorgos Seferis']


# Doing our share of data validation
In our Nobel prizes collection, each document has an array of laureate subdocuments "laureates", each containing information such as the prize share for a laureate:

```
{'_id': ObjectId('5bc56145f35b634065ba1997'),
 'category': 'chemistry',
 'laureates': [{'firstname': 'Frances H.',
   'id': '963',
   'motivation': '"for the directed evolution of enzymes"',
   'share': '2',
   'surname': 'Arnold'},
  {'firstname': 'George P.',
   'id': '964',
   'motivation': '"for the phage display of peptides and antibodies"',
   'share': '4',
   'surname': 'Smith'},
 {...
```

Each "laureates.share" value appears to be the reciprocal of a laureate's fractional share of that prize, encoded as a string. For example, a laureate "share" of "4" means that this laureate received a $\frac{1}{4}$ share of the prize. Let's check that for each prize, all the shares of all the laureates add up to 1!

Notice the quotes around the values in the "share" field: these values are actually given as strings! You'll have to convert then to numbers before you find the reciprocals and add up the shares.

In [13]:
# Save documents, projecting out laureates share
prizes = db.prizes.find({}, ["laureates.share"])

# Iterate over prizes
for prize in prizes:
    # Initialize total share
    total_share = 0

    # Iterate over laureates for the prize
    for laureate in prize["laureates"]:
        # add the share of the laureate to total_share
        total_share += 1 / float(laureate['share'])

    # Print the total share
    print(total_share)  

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0


# What the sort?
This block prints out the first five projections of a sorted query. What "sort" argument fills the blank?

```
docs = list(db.laureates.find(
    {"born": {"$gte": "1900"}, "prizes.year": {"$gte": "1954"}},
    {"born": 1, "prizes.year": 1, "_id": 0},
    sort=____))
for doc in docs[:5]:
    print(doc)
```

```
{'born': '1916-08-25', 'prizes': [{'year': '1954'}]}
{'born': '1915-06-15', 'prizes': [{'year': '1954'}]}
{'born': '1901-02-28', 'prizes': [{'year': '1954'}, {'year': '1962'}]}
{'born': '1913-07-12', 'prizes': [{'year': '1955'}]}
{'born': '1911-01-26', 'prizes': [{'year': '1955'}]}
```

In [16]:
docs = list(db.laureates.find(
    {"born": {"$gte": "1900"}, "prizes.year": {"$gte": "1954"}},
    {"born": 1, "prizes.year": 1, "_id": 0},
    sort=[("prizes.year", 1), ("born", -1)]))
for doc in docs[:5]:
    print(doc)

{'born': '1916-08-25', 'prizes': [{'year': '1954'}]}
{'born': '1915-06-15', 'prizes': [{'year': '1954'}]}
{'born': '1901-02-28', 'prizes': [{'year': '1954'}, {'year': '1962'}]}
{'born': '1913-07-12', 'prizes': [{'year': '1955'}]}
{'born': '1911-01-26', 'prizes': [{'year': '1955'}]}


In [17]:
docs = list(db.laureates.find(
    {"born": {"$gte": "1900"}, "prizes.year": {"$gte": "1954"}},
    {"born": 1, "prizes.year": 1, "_id": 0},
    sort={"prizes.year": 1, "born": -1}))
for doc in docs[:5]:
    print(doc)

TypeError: passing a dict to sort/create_index/hint is not allowed - use a list of tuples instead. did you mean [('prizes.year', 1), ('born', -1)]?

# Sorting together: MongoDB + Python
In this exercise you'll explore the prizes in the physics category.You will use Python to sort laureates for one prize by last name, and then MongoDB to sort prizes by year:

```
1901: Röntgen
1902: Lorentz and Zeeman
1903: Becquerel and Curie and Curie, née Sklodowska
```

You'll start by writing a function that takes a prize document as an argument, extracts all the laureates from that document, arranges them in alphabetical order, and returns a string containing the last names separated by " and ".

The Nobel database is again available to you as db. We also pre-loaded a sample document sample_doc so you can test your laureate-extracting function.

(Remember that you can always type help(function_name) in console to get a refresher on functions you might be less familiar with, e.g. help(sorted)!)

In [24]:
print(db.prizes.find_one())

{'_id': ObjectId('6018022985c61d018cc9827d'), 'year': '2018', 'category': 'physics', 'overallMotivation': '“for groundbreaking inventions in the field of laser physics”', 'laureates': [{'id': '960', 'firstname': 'Arthur', 'surname': 'Ashkin', 'motivation': '"for the optical tweezers and their application to biological systems"', 'share': '2'}, {'id': '961', 'firstname': 'Gérard', 'surname': 'Mourou', 'motivation': '"for their method of generating high-intensity, ultra-short optical pulses"', 'share': '4'}, {'id': '962', 'firstname': 'Donna', 'surname': 'Strickland', 'motivation': '"for their method of generating high-intensity, ultra-short optical pulses"', 'share': '4'}]}


In [25]:
from operator import itemgetter

sample_prize = db.prizes.find_one()

def all_laureates(prize):  
  # sort the laureates by surname
  sorted_laureates = sorted(prize['laureates'], key=itemgetter('surname'))
  
  # extract surnames
  surnames = [laureate["surname"] for laureate in sorted_laureates]
  
  # concatenate surnames separated with " and " 
  all_names = " and ".join(surnames)
  
  return all_names

# test the function on a sample doc
print(all_laureates(sample_prize))

Ashkin and Mourou and Strickland


In [26]:
from operator import itemgetter

def all_laureates(prize):  
  # sort the laureates by surname
  sorted_laureates = sorted(prize["laureates"], key=itemgetter("surname"))
  
  # extract surnames
  surnames = [laureate["surname"] for laureate in sorted_laureates]
  
  # concatenate surnames separated with " and " 
  all_names = " and ".join(surnames)
  
  return all_names

# find physics prizes, project year and first and last name, and sort by year
docs = db.prizes.find(
           filter={"category": "physics"},
           projection=["year", "laureates.firstname", "laureates.surname"],
           sort=[("year", 1)])

In [27]:
from operator import itemgetter

def all_laureates(prize):  
  # sort the laureates by surname
  sorted_laureates = sorted(prize["laureates"], key=itemgetter("surname"))
  
  # extract surnames
  surnames = [laureate["surname"] for laureate in sorted_laureates]
  
  # concatenate surnames separated with " and " 
  all_names = " and ".join(surnames)
  
  return all_names

# find physics prizes, project year and name, and sort by year
docs = db.prizes.find(
           filter= {"category": "physics"}, 
           projection= ["year", "laureates.firstname", "laureates.surname"], 
           sort= [("year", 1)])

# print the year and laureate names (from all_laureates)
for doc in docs:
    print("{year}: {names}".format(year=doc['year'], names=all_laureates(doc)))

1901: Röntgen
1902: Lorentz and Zeeman
1903: Becquerel and Curie and Curie, née Sklodowska
1904: (John William Strutt)
1905: von Lenard
1906: Thomson
1907: Michelson
1908: Lippmann
1909: Braun and Marconi
1910: van der Waals
1911: Wien
1912: Dalén
1913: Kamerlingh Onnes
1914: von Laue
1915: Bragg and Bragg
1917: Barkla
1918: Planck
1919: Stark
1920: Guillaume
1921: Einstein
1922: Bohr
1923: Millikan
1924: Siegbahn
1925: Franck and Hertz
1926: Perrin
1927: Compton and Wilson
1928: Richardson
1929: de Broglie
1930: Raman
1932: Heisenberg
1933: Dirac and Schrödinger
1935: Chadwick
1936: Anderson and Hess
1937: Davisson and Thomson
1938: Fermi
1939: Lawrence
1943: Stern
1944: Rabi
1945: Pauli
1946: Bridgman
1947: Appleton
1948: Blackett
1949: Yukawa
1950: Powell
1951: Cockcroft and Walton
1952: Bloch and Purcell
1953: Zernike
1954: Born and Bothe
1955: Kusch and Lamb
1956: Bardeen and Brattain and Shockley
1957: Lee and Yang
1958: Cherenkov and Frank and Tamm
1959: Chamberlain and Segrè
19

# Gap years
The prize in economics was not added until 1969. There have also been many years for which prizes in one or more of the original categories were not awarded.

In this exercise, you will utilize sorting by multiple fields to see which categories are missing in which years.

For now, you will just print the list of all documents, but in the next chapter, you'll learn how to use MongoDB to group and aggregate data to present this information in a more convenient format.

In [28]:
# original categories from 1901
original_categories = db.prizes.distinct('category', {'year': '1901'})
print(original_categories)

# project year and category, and sort
docs = db.prizes.find(
    filter={},
    projection={'year': 1, 'category': 1, "_id": 0},
    sort=[('year', -1), ('category', 1)]
)

# print the documents
for doc in docs:
    print(doc)

['chemistry', 'literature', 'medicine', 'peace', 'physics']
{'year': '2018', 'category': 'chemistry'}
{'year': '2018', 'category': 'economics'}
{'year': '2018', 'category': 'medicine'}
{'year': '2018', 'category': 'peace'}
{'year': '2018', 'category': 'physics'}
{'year': '2017', 'category': 'chemistry'}
{'year': '2017', 'category': 'economics'}
{'year': '2017', 'category': 'literature'}
{'year': '2017', 'category': 'medicine'}
{'year': '2017', 'category': 'peace'}
{'year': '2017', 'category': 'physics'}
{'year': '2016', 'category': 'chemistry'}
{'year': '2016', 'category': 'economics'}
{'year': '2016', 'category': 'literature'}
{'year': '2016', 'category': 'medicine'}
{'year': '2016', 'category': 'peace'}
{'year': '2016', 'category': 'physics'}
{'year': '2015', 'category': 'chemistry'}
{'year': '2015', 'category': 'economics'}
{'year': '2015', 'category': 'literature'}
{'year': '2015', 'category': 'medicine'}
{'year': '2015', 'category': 'peace'}
{'year': '2015', 'category': 'physics'}

# High-share categories
In the year 3030, everybody wants to be a Nobel laureate. Over the last thousand years, many new categories have been added. You serve a MongoDB prizes collection with the same schema as we've seen. Many people theorize that they have a better chance in "high-share" categories. They are hitting your server with similar, long-running queries. It's time to cover those queries with an index.

Which of the following indexes is best suited to speeding up the operation 
```
db.prizes.distinct("category", {"laureates.share": {"$gt": "3"}})?
```

In [29]:
print(db.prizes.distinct("category", {"laureates.share": {"$gt": "3"}}))

['chemistry', 'medicine', 'physics']


# Recently single?
A prize might be awarded to a single laureate or to several. For each prize category, report the most recent year that a single laureate -- rather than several -- received a prize in that category. As part of this task, you will ensure an index that speeds up finding prizes by category and then sorting results by decreasing year

In [30]:
# Specify an index model for compound sorting
index_model = [('category', 1), ('year', -1)]
db.prizes.create_index(index_model)

# Collect the last single-laureate year for each category
report = ""
for category in sorted(db.prizes.distinct("category")):
    doc = db.prizes.find_one(
        {'category': category, "laureates.share": "1"},
        sort=[('year', -1)]
    )
    report += "{category}: {year}\n".format(**doc)

print(report)

chemistry: 2011
economics: 2017
literature: 2017
medicine: 2016
peace: 2017
physics: 1992



# Born and affiliated
Some countries are, for one or more laureates, both their country of birth ("bornCountry") and a country of affiliation for one or more of their prizes ("prizes.affiliations.country"). You will find the five countries of birth with the highest counts of such laureates.

In [31]:
from collections import Counter

# Ensure an index on country of birth
db.laureates.create_index([('bornCountry', 1)])

# Collect a count of laureates for each country of birth
n_born_and_affiliated = {
    country: db.laureates.count_documents({
        'bornCountry': country,
        "prizes.affiliations.country": country
    })
    for country in db.laureates.distinct("bornCountry")
}

five_most_common = Counter(n_born_and_affiliated).most_common(5)
print(five_most_common)

[('USA', 241), ('United Kingdom', 56), ('France', 26), ('Germany', 19), ('Japan', 17)]


# Setting a new limit?
How many documents does the following expression return?

```
list(db.prizes.find({"category": "economics"},
                    {"year": 1, "_id": 0})
     .sort("year")
     .limit(3)
     .limit(5))
```

In [32]:
list(db.prizes.find({"category": "economics"},
                    {"year": 1, "_id": 0})
     .sort("year")
     .limit(3)
     .limit(5))

[{'year': '1969'},
 {'year': '1970'},
 {'year': '1971'},
 {'year': '1972'},
 {'year': '1973'}]

# The first five prizes with quarter shares
Find the first five prizes with one or more laureates sharing 1/4 of the prize. Project our prize category, year, and laureates' motivations.

In [33]:
from pprint import pprint

# Fetch prizes with quarter-share laureate(s)
filter_ = {'laureates.share': '4'}

# Save the list of field names
projection = ['category', 'year', 'laureates.motivation']

# Save a cursor to yield the first five prizes
cursor = db.prizes.find(filter_, projection).sort('year').limit(5)
pprint(list(cursor))

[{'_id': ObjectId('6018022985c61d018cc98472'),
  'category': 'physics',
  'laureates': [{'motivation': '"in recognition of the extraordinary services '
                               'he has rendered by his discovery of '
                               'spontaneous radioactivity"'},
                {'motivation': '"in recognition of the extraordinary services '
                               'they have rendered by their joint researches '
                               'on the radiation phenomena discovered by '
                               'Professor Henri Becquerel"'},
                {'motivation': '"in recognition of the extraordinary services '
                               'they have rendered by their joint researches '
                               'on the radiation phenomena discovered by '
                               'Professor Henri Becquerel"'}],
  'year': '1903'},
 {'_id': ObjectId('6018022985c61d018cc98419'),
  'category': 'chemistry',
  'laureates': [{'motivation':

# Pages of particle-prized people
You and a friend want to set up a website that gives information on Nobel laureates with awards relating to particle phenomena. You want to present these laureates one page at a time, with three laureates per page. You decide to order the laureates chronologically by award year. When there is a "tie" in ordering (i.e. two laureates were awarded prizes in the same year), you want to order them alphabetically by surname.

In [34]:
from pprint import pprint

# Write a function to retrieve a page of data
def get_particle_laureates(page_number=1, page_size=3):
    if page_number < 1 or not isinstance(page_number, int):
        raise ValueError("Pages are natural numbers (starting from 1).")
    particle_laureates = list(
        db.laureates.find(
            {'prizes.motivation': {'$regex': "particle"}},
            ["firstname", "surname", "prizes"])
        .sort([('prizes.year', 1), ('surname', 1)])
        .skip(page_size * (page_number - 1))
        .limit(page_size))
    return particle_laureates


# Collect and save the first nine pages
pages = [get_particle_laureates(page_number=page) for page in range(1, 9)]
pprint(pages[0])


[{'_id': ObjectId('6018024085c61d018cc9852a'),
  'firstname': 'Charles Thomson Rees',
  'prizes': [{'affiliations': [{'city': 'Cambridge',
                                'country': 'United Kingdom',
                                'name': 'University of Cambridge'}],
              'category': 'physics',
              'motivation': '"for his method of making the paths of '
                            'electrically charged particles visible by '
                            'condensation of vapour"',
              'share': '2',
              'year': '1927'}],
  'surname': 'Wilson'},
 {'_id': ObjectId('6018024085c61d018cc98540'),
  'firstname': 'Sir John Douglas',
  'prizes': [{'affiliations': [{'city': 'Harwell, Berkshire',
                                'country': 'United Kingdom',
                                'name': 'Atomic Energy Research '
                                        'Establishment'}],
              'category': 'physics',
              'motivation': '"for their pione